I this Jupyter notebook, we will load the data and retrieve the number of palms types existing to know how to divide the dataset.

In [3]:
import geopandas as gpd
import os
import json
import cv2
from tqdm import tqdm
import random

We will count how many palms exists per type category in GC, TNF and La Gomera

In [4]:
# Ruta del archivo Shapefile con las coordenadas de las palmeras
shapefile_path = r"F:\Universidad\Curso 2024-25\Segundo Semestre\TFG\Desarrollo\Mapa_IDECanarias_Palmeras/99_K_Mapapalmerascanarias.shp"

# Cargar y leer el archivo
data = gpd.read_file(shapefile_path)

In [5]:
# Filtrar solo las palmeras de Gran Canaria
data_gc_tnf_gm = data[data["ISLA"].isin(["GRAN CANARIA", "TENERIFE", "LA GOMERA"])].copy()

# Convertimos los valores None de la columna "Hibrid" a PC (Palmera Canaria), que quiere decir que no se presenta hibridación
data_gc_tnf_gm["Hibrid"] = data_gc_tnf_gm["Hibrid"].fillna("pc")

# Contar el número de palmeras por tipo de ambiente
palm_type_counts = data_gc_tnf_gm.value_counts("Hibrid", ascending=True)

# Eliminamos del dataframe las palmeras con hibridaje de dificil asignación o "?"
data_gc_tnf_gm = data_gc_tnf_gm[data_gc_tnf_gm["Hibrid"] != "?"]

print(data_gc_tnf_gm.head(10))

# Mostrar los resultados
print("Recuento de palmeras por tipo en Gran Canaria, Tenerife y La Gomera:")
print(palm_type_counts)

                 Id_palm Tip_amb                       Tip_amb_de Hibrid  \
132957  Cx454646y3100878       ?  Ambientes de difícil asignación     pc   
132958  Cx454653y3100874       ?  Ambientes de difícil asignación     pc   
132959  Cx454659y3100870       ?  Ambientes de difícil asignación     pc   
132960  Cx454677y3100862       ?  Ambientes de difícil asignación     pc   
132961  Cx454672y3100865       ?  Ambientes de difícil asignación     pc   
132962  Cx454666y3100868       ?  Ambientes de difícil asignación     pc   
132963  Cx454607y3100765       ?  Ambientes de difícil asignación     pc   
132964  Cx454547y3100787       ?  Ambientes de difícil asignación     pc   
132965  Cx454758y3100800       ?  Ambientes de difícil asignación     pc   
132966  Cx454745y3100790       ?  Ambientes de difícil asignación     pc   

                Hibrid_de          ISLA      MUNICIPIO  \
132957  Palmeras canarias  GRAN CANARIA  SANTA BRÍGIDA   
132958  Palmeras canarias  GRAN CANARIA  SANTA 

Entrenamiento y validación del modelo de YOLO-cls para clasificación de tipos de palmeras.

In [ ]:
from ultralytics import YOLO

# Cargamos el modelo YOLOv11 de clasificación preentrenado
model = YOLO("yolo11m-cls.pt")

# Entrenamos el modelo con las imágenes clasificadas por ambiente
model.train(
    data=r"F:\Universidad\Curso 2024-25\Segundo Semestre\TFG\Desarrollo\dataset\classification_per_type\palm_type_classification_dataset",
    epochs=100,
    patience=10,
    imgsz=32,
    batch=64,
    device=0,
    plots = True,
    name="yolo_type_train",
    project=r".\classification\type\yolov11m",
    # Mejora de generalización con estos parámetros
    label_smoothing=0.1,     # Suavizado de etiquetas
    weight_decay=1e-4,       # Penalización L2
    freeze=[0, 1, 2],        # Congela capas backbone más profundas
    mixup=0.2,               # Aplica mixup con probabilidad 20%
    cutmix=0.2,              # Aplica cutmix con probabilidad 20%
    augment=True             # Aplica aumentos por defecto
)

In [ ]:
model_path = r".\classification\env\yolov11m\yolo_type_train\weights\best.pt"
validation_path = r"F:\Universidad\Curso 2024-25\Segundo Semestre\TFG\Desarrollo\dataset\classification_per_type\palm_type_classification_dataset"
model = YOLO(model_path)
metrics = model.val(data=validation_path, split="val", project=r".\classification\type\yolov11m", name="yolo_type_val", plots=True)

Entrenamiento y validación del modelo de ResNet50 para clasificación de tipos de palmeras.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

dataset_dir = r"F:\Universidad\Curso 2024-25\Segundo Semestre\TFG\Desarrollo\dataset\classification_per_type\palm_type_classification_dataset"
dir_to_save_metrics = r"classification\type\resnet50\metrics"
dir_to_save_model = r"classification\type\resnet50\resnet50_best_final.pth"
num_classes = len(os.listdir(os.path.join(dataset_dir, 'train')))
batch_size = 32
num_epochs = 50
lr = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parametros para Early Stopping
patience = 10
epochs_without_improvement = 0
best_val_acc = 0.0

# Transforms para entrenamiento con data augmentation
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor()
])

# Transformador para validación
transform_val = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Cargar datos
train_dataset = datasets.ImageFolder(os.path.join(dataset_dir, 'train'), transform=transform_train)
val_dataset = datasets.ImageFolder(os.path.join(dataset_dir, 'val'), transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Cargar modelo preentrenado
model = models.resnet50(pretrained=True)

# Congelar capas convolucionales
for param in model.parameters():
    param.requires_grad = False

# Descongelar capa 'layer4'
for name, param in model.named_parameters():
    if "layer4" in name:
        param.requires_grad = True

# Regularización para evitar el sobreajuste
model.fc = nn.Sequential(
    nn.Dropout(p=0.6),  # 60% de las neuronas se apagan aleatoriamente durante el entrenamiento
    nn.Linear(model.fc.in_features, num_classes)
)

model = model.to(device)


# Optimizador y scheduler que ajusta tasa de aprendizaje a medida que avanza el entrenamiento
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)
# Suavizado de etiquetas para evitar el sobreajuste
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  

# Métricas por época
train_losses, val_losses = [], []
val_accuracies, val_f1s, val_precisions, val_recalls = [], [], [], []

best_val_acc = 0.0
epochs_without_improvement = 0

for epoch in range(num_epochs):

    model.train()
    train_loss = 0.0

    # Entrenamiento por lotes
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validación
    model.eval()
    val_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    scheduler.step(avg_val_loss)

    # Cálculo de métricas
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')

    val_accuracies.append(acc)
    val_f1s.append(f1)
    val_precisions.append(precision)
    val_recalls.append(recall)

    cm = confusion_matrix(all_labels, all_preds)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | "
          f"Val Acc: {acc:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")

    # Early stopping
    if acc > best_val_acc:
        best_val_acc = acc
        torch.save(model.state_dict(), dir_to_save_model)
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("Early stopping")
            break

# Cargar el mejor modelo guardado
print("Cargando el modelo:")
model.load_state_dict(torch.load(dir_to_save_model))
model.eval()

# Evaluación final en validación
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Métricas finales
final_acc = accuracy_score(all_labels, all_preds)
final_f1 = f1_score(all_labels, all_preds, average='macro')
final_precision = precision_score(all_labels, all_preds, average='macro')
final_recall = recall_score(all_labels, all_preds, average='macro')

print("Evaluación final del mejor modelo guardado:")
print(f"   Accuracy:  {final_acc:.4f}")
print(f"   F1 Macro:  {final_f1:.4f}")
print(f"   Precision: {final_precision:.4f}")
print(f"   Recall:    {final_recall:.4f}")


if not os.path.exists(dir_to_save_metrics):
    os.makedirs(dir_to_save_metrics)

# Guardar métricas
df_log = pd.DataFrame({
    'epoch': list(range(1, len(train_losses)+1)),
    'train_loss': train_losses,
    'val_loss': val_losses,
    'val_accuracy': val_accuracies,
    'val_f1_macro': val_f1s,
    'val_precision': val_precisions,
    'val_recall': val_recalls
})
df_log.to_csv(os.path.join(dir_to_save_metrics, "training_metrics_log.csv"), index=False)

# Graficar métricas
plt.figure()
plt.plot(df_log['epoch'], df_log['train_loss'], label='Train Loss')
plt.plot(df_log['epoch'], df_log['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(dir_to_save_metrics, "loss_curve.png"))
plt.close()

plt.figure()
plt.plot(df_log['epoch'], df_log['val_accuracy'], label='Validation Accuracy')
plt.plot(df_log['epoch'], df_log['val_f1_macro'], label='F1 Macro')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Validation Accuracy & F1 Macro Over Epochs')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(dir_to_save_metrics, "accuracy_f1_curve.png"))
plt.close()

class_names = val_dataset.classes
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.savefig(os.path.join(dir_to_save_metrics, "confusion_matrix.png"))
plt.close()

# Matriz de confusión normalizada (por fila)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap='Blues', cbar=True, xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (Normalized by True Labels)")
plt.savefig(os.path.join(dir_to_save_metrics, "confusion_matrix_normalized.png"))
plt.close()

Entrenamiento y validación con MobileNet para la clasificación de tipos de palmeras.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

# Configuración
dataset_dir = r"F:\\Universidad\\Curso 2024-25\\Segundo Semestre\\TFG\\Desarrollo\\dataset\\classification_per_type\\palm_type_classification_dataset"
dir_to_save_metrics = r"classification\type\mobilenet\metrics"
dir_to_save_model = r"classification\type\mobilenet\mobilenet.pth"
os.makedirs(dir_to_save_metrics, exist_ok=True)
num_classes = len(os.listdir(os.path.join(dataset_dir, 'train')))
batch_size = 32
num_epochs = 100
lr = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
patience = 10

# Tranformadores para el entrenamiento (con data augmentation) y validación
transform_train = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
    transforms.GaussianBlur(kernel_size=3),
    transforms.ToTensor(),
])

transform_val = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
])

# Datasets y Loaders
train_dataset = datasets.ImageFolder(os.path.join(dataset_dir, 'train'), transform=transform_train)
val_dataset = datasets.ImageFolder(os.path.join(dataset_dir, 'val'), transform=transform_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
class_names = val_dataset.classes

# Pesos de clase
labels = [sample[1] for sample in train_dataset.samples]
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)


# Modelo MobileNet
model = models.mobilenet_v2(pretrained=True)

# Congelar solo capas iniciales
for name, param in model.features.named_parameters():
    if int(name.split('.')[0]) < 5:
        param.requires_grad = False

model.classifier = nn.Sequential(
    nn.Dropout(0.2),    # Dropout del 20% para evitar el sobreajuste
    nn.Linear(model.last_channel, num_classes)
)
model = model.to(device)

# Optimizador y scheduler que ajusta tasa de aprendizaje a medida que avanza el entrenamiento
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)
# Función de pérdida
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# Métricas
train_losses, val_losses = [], []
val_accuracies, val_f1s, val_precisions, val_recalls = [], [], [], []
best_val_acc = 0.0
epochs_without_improvement = 0

# Entrenamiento por épocas
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    val_loss = 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    scheduler.step(avg_val_loss)

    # Cálculo de métricas
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    val_accuracies.append(acc)
    val_f1s.append(f1)
    val_precisions.append(precision)
    val_recalls.append(recall)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | "
          f"Val Acc: {acc:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")

    if acc > best_val_acc:
        best_val_acc = acc
        torch.save(model.state_dict(), dir_to_save_model)
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("Early stopping")
            break

# Evaluación final
print("Cargando el mejor modelo guardado...")
model.load_state_dict(torch.load(dir_to_save_model))
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

final_acc = accuracy_score(all_labels, all_preds)
final_f1 = f1_score(all_labels, all_preds, average='macro')
final_precision = precision_score(all_labels, all_preds, average='macro')
final_recall = recall_score(all_labels, all_preds, average='macro')
print("Evaluación final:")
print(f"   Accuracy:  {final_acc:.4f}")
print(f"   F1 macro:  {final_f1:.4f}")
print(f"   Precision: {final_precision:.4f}")
print(f"   Recall:    {final_recall:.4f}")

# Guardado de métricas
if not os.path.exists(dir_to_save_metrics):
    os.makedirs(dir_to_save_metrics)

df_log = pd.DataFrame({
    'epoch': list(range(1, len(train_losses)+1)),
    'train_loss': train_losses,
    'val_loss': val_losses,
    'val_accuracy': val_accuracies,
    'val_f1_macro': val_f1s,
    'val_precision': val_precisions,
    'val_recall': val_recalls
})
df_log.to_csv(os.path.join(dir_to_save_metrics, "training_metrics_log.csv"), index=False)


# Gráficas
plt.figure()
plt.plot(df_log['epoch'], df_log['train_loss'], label='Train Loss')
plt.plot(df_log['epoch'], df_log['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(dir_to_save_metrics, "loss_curve.png"))
plt.close()

plt.figure()
plt.plot(df_log['epoch'], df_log['val_accuracy'], label='Validation Accuracy')
plt.plot(df_log['epoch'], df_log['val_f1_macro'], label='F1 macro')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Validation Accuracy & F1 macro Over Epochs')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(dir_to_save_metrics, "accuracy_f1_curve.png"))
plt.close()

# Matriz de confusión
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.savefig(os.path.join(dir_to_save_metrics, "confusion_matrix.png"))
plt.close()

# Matriz normalizada
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (Normalized)")
plt.savefig(os.path.join(dir_to_save_metrics, "confusion_matrix_normalized.png"))
plt.close()